In [1]:
!pip install transformers

     |████████████████████████████████| 7.4 MB 5.6 MB/s            
  Using cached tokenizers-0.13.3-cp38-cp38-macosx_12_0_arm64.whl (3.9 MB)
     |████████████████████████████████| 401 kB 6.0 MB/s            
     |████████████████████████████████| 288 kB 6.3 MB/s            


In [17]:
import csv

example_text = 'Renewable energy sources, such as solar, wind, and hydroelectric power, are gaining traction as viable alternatives to fossil fuels. These sources harness naturally available resources, reducing our reliance on finite and environmentally harmful energy sources. Solar energy, for instance, utilizes photovoltaic panels to convert sunlight into electricity. Wind energy harnesses the power of wind turbines to produce clean and sustainable electricity. Hydroelectric power, on the other hand, generates electricity by utilizing the force of flowing water. The widespread adoption of renewable energy technologies can mitigate climate change by reducing greenhouse gas emissions. Government initiatives and incentives are encouraging the transition towards renewable energy, offering tax credits and grants to individuals and businesses. Additionally, investment in research and development is driving innovation in this sector, leading to more efficient and cost-effective renewable energy solutions. As renewable energy infrastructures expand, job creation in the green energy sector is expected to rise, stimulating economic growth. Moreover, decentralized renewable energy systems can improve energy access for remote and underserved communities. The integration of renewable energy into the power grid is enhancing grid stability and resilience while providing consumers with affordable and sustainable electricity options.'

example_texts = [example_text for _ in range(100)]

with open('test_file.csv', 'w', newline='') as csv_output:
        fieldnames = ['text']

        # Create a CSV writer object
        csv_writer = csv.DictWriter(csv_output, fieldnames=fieldnames)

        # Write the header
        csv_writer.writeheader()

        for ex_txt in example_texts:

            # Write the combined data for the current row
            csv_writer.writerow({'text': ex_txt})


csv_output.close()




In [19]:
import csv

def get_first_and_remaining_sentences(text, num_first):
    sentences = text.split('. ')
    first_three_sentences = '. '.join(sentences[:num_first]) + '.'
    remaining_sentences = '. '.join(sentences[num_first:]) + '.'
    return first_three_sentences, remaining_sentences


# Open the input CSV file
with open('test_file.csv', 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)

    # Open the output CSV file for writing
    with open('test_properly_formatted_data.csv', 'w', newline='') as csv_output:
        fieldnames = ['input', 'target']

        # Create a CSV writer object
        csv_writer = csv.DictWriter(csv_output, fieldnames=fieldnames)

        # Write the header
        csv_writer.writeheader()

        # Loop through the rows, combine the author and quote, and write to the new CSV file
        for row in csv_reader:
            ex_text = row['text']
            first_k_sentences, remaining_sentences = get_first_and_remaining_sentences(ex_text, num_first=3)

            # Write the combined data for the current row
            csv_writer.writerow({'input': ex_text, 'target': remaining_sentences})

    csv_output.close()



In [1]:
from custom_dataset import customData
import datasets
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = 'daryl149/Llama-2-7b-hf'
# Pull the tokenizer.
tokenizer = AutoTokenizer.from_pretrained(model_id)

dataset = customData(tokenizer=tokenizer, csv_name='test_properly_formatted_data.csv')
#from datasets import load_dataset

#import pandas as pd

#df = pd.read_csv('test_properly_formatted_data.csv')
#dataset = Dataset.from_pandas(df)

#load_dataset(
#                "csv",
#                data_files='/Users/bradlystadie/Documents/GitHub/three-cases/test_properly_formatted_data.csv',  # "eval": "grammar_validation.csv"},
#                delimiter=",",
#            )

#data_files = {"train": "test_properly_formatted_data.csv", "test": "test_properly_formatted_data.csv"}
#dataset = load_dataset("namespace/your_dataset_name", data_files=data_files)

# dataset = load_dataset("csv", data_files="test_properly_formatted_data.csv")


# dataset = customData(tokenizer=tokenizer, csv_name='test_properly_formatted_data.csv')

# dataset.save_to_disk("test_dataset.hf")


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


# This is a library that can just pull datasets. They're expected to be in a standard format.
data = load_dataset("Abirate/english_quotes")

# LLAMA weights.
model_id = 'daryl149/Llama-2-7b-hf'
# Pull the tokenizer.
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Apply the tokenizer to each sample in the dataset.
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

# data has train and split.
one_data = data['train'][0]

In [ ]:
import datasets
from utils import Concatenator

def get_preprocessed_samsum(dataset_config, tokenizer, split):
    dataset = datasets.load_dataset("samsum", split=split)



    # this trick of defining a formatting and then calling prompt.format(key_1=string_1)
    # is very useful if you have multiple keys that you need to
    # compress into a single string.
    # for example, in this case we want to take a dialogue and
    # learn a model that produces a summary.
    # We can define a prompt that expects to get the dialogue
    # and the summary as input
    # and then also adds the flavor text at the start and the eos token at the end.
    prompt = (
        f"Summarize this dialog:\n{{dialog}}\n---\nSummary:\n{{summary}}{{eos_token}}"
    )

    # this function actually applies the prompt to the given sample.
    # You can then call this with a map function.
    def apply_prompt_template(sample):
        return {
            "text": prompt.format(
                dialog=sample["dialogue"],
                summary=sample["summary"],
                eos_token=tokenizer.eos_token,
            )
        }

    dataset = dataset.map(apply_prompt_template, remove_columns=list(dataset.features))

    # This tokenizes the data after putting it into the correct prompt format.
    # The line at the end .map(Concatenator()) basically
    # takes multiple strings with the EOS token at the end of each string.
    # it then concats them together.
    # this helps with training efficiency, since now each of your inputs
    # to the model are guaranteed to have the same size.
    dataset = dataset.map(
        lambda sample: tokenizer(sample["text"]),
        batched=True,
        remove_columns=list(dataset.features),
    ).map(Concatenator(), batched=True)
    return dataset